In [12]:
from pyomo.environ import *
import numpy as np
import time
import json
from scipy.interpolate import RegularGridInterpolator

In [7]:
96 * 48

4608

In [18]:
EMB, HEAD, QK, V, MLP = 768,12,64,64,3072
all_variable_specs = {
    "EMB": [768, 32, 768//32+1],
    "HEAD": [12, 1, 12//1+1],
    "QK": [64, 2, 64//2+1],
    "V": [64, 2, 64//2+1],
    "MLP": [3072, 32, 3072//32+1],
}

In [19]:
latency_file = "latency_head.json"
with open(latency_file) as json_file:
    measurement = json.load(json_file)
EMB = np.arange(4)*256
head = np.array([1,3,6,9,12])
QK = np.array([1,16,32,48,64])
V = np.array([1,16,32,48,64])
MLP = (np.arange(25))*128
data = np.zeros([4,5,5,5,25])
for i in range(3):
    for j in range(5):
        for k1 in range(5):
            for k2 in range(5):
                for l in range(25):
                    e = EMB[i+1]
                    q_h = head[j]
                    q = QK[k1]
                    v = V[k2]
                    if MLP[l]:
                        h = MLP[l]
                    else:
                        h = 1
                    data[i+1,j,k1,k2,l] = measurement['EMB_'+str(e)]['QK_'+str(q_h)+'_'+str(q)]['V_'+str(v)]['MLP_'+str(h)]
latency_look_up_table = RegularGridInterpolator((EMB, head, QK, V, MLP), data) 

In [28]:
# Define index ranges and strides
emb_idx_range = np.arange(all_variable_specs["EMB"][2])
head_idx_range = np.arange(all_variable_specs["HEAD"][2])
qk_idx_range = np.arange(all_variable_specs["QK"][2])
v_idx_range = np.arange(all_variable_specs["V"][2])
mlp_idx_range = np.arange(all_variable_specs["MLP"][2])

# Calculate and fill latency table
for emb_idx in emb_idx_range:
    print(f"progress {emb_idx}/{all_variable_specs['EMB'][2]}")
    
    emb_values = np.maximum(emb_idx * all_variable_specs["EMB"][1], 1)
    head_values = np.maximum(head_idx_range * all_variable_specs["HEAD"][1], 1)
    qk_values = np.maximum(qk_idx_range * all_variable_specs["QK"][1], 1)
    v_values = np.maximum(v_idx_range * all_variable_specs["V"][1], 1)
    mlp_values = np.maximum(mlp_idx_range * all_variable_specs["MLP"][1], 1)
    
    emb_grid, head_grid, qk_grid, v_grid, mlp_grid = np.meshgrid(emb_values, head_values, qk_values, v_values, mlp_values, indexing='ij')

    dim_vecs = np.vstack([emb_grid.ravel(), head_grid.ravel(), qk_grid.ravel(), v_grid.ravel(), mlp_grid.ravel()]).T
    latency_values = latency_look_up_table(dim_vecs)
    
    latency_table[emb_idx, :, :, :, :] = latency_values.reshape(all_variable_specs["HEAD"][2], all_variable_specs["QK"][2], all_variable_specs["V"][2], all_variable_specs["MLP"][2])

progress 0/25
progress 1/25
progress 2/25
progress 3/25
progress 4/25
progress 5/25
progress 6/25
progress 7/25
progress 8/25
progress 9/25
progress 10/25
progress 11/25
progress 12/25
progress 13/25
progress 14/25
progress 15/25
progress 16/25
progress 17/25
progress 18/25
progress 19/25
progress 20/25
progress 21/25
progress 22/25
progress 23/25
progress 24/25


In [33]:
test_emb = 6
test_head = 11
test_qk = 16
test_v = 30
test_mlp = 40

# Convert test dimensions to the scale of the data
test_dim_vec = np.array([
    max(test_emb * all_variable_specs["EMB"][1], 1),
    max(test_head * all_variable_specs["HEAD"][1], 1),
    max(test_qk * all_variable_specs["QK"][1], 1),
    max(test_v * all_variable_specs["V"][1], 1),
    max(test_mlp * all_variable_specs["MLP"][1], 1)
])

# Print the latency table value at the specified indices
print(latency_table[test_emb, test_head, test_qk, test_v, test_mlp])

# Interpolate to find the latency value
latency_value = latency_look_up_table(test_dim_vec)
print(latency_value)

0.025570660829544067
[0.02557066]


In [35]:
import pickle
with open("minlp_latency_table.pkl", "wb") as f:
    pickle.dump(latency_table, f)

In [29]:
# latency_table_shape = tuple([x[2] for x in all_variable_specs.values()])
# latency_table = np.zeros(latency_table_shape)

# for emb_idx in range(all_variable_specs["EMB"][2]):
#     print(f"progress {emb_idx}/{all_variable_specs['EMB'][2]}")
#     for head_idx in range(all_variable_specs["HEAD"][2]):
#         for qk_idx in range(all_variable_specs["QK"][2]):
#             for v_idx in range(all_variable_specs["V"][2]):
#                 for mlp_idx in range(all_variable_specs["MLP"][2]):
                    
#                     emb = max(emb_idx*all_variable_specs["EMB"][1], 1)
#                     head = max(head_idx*all_variable_specs["HEAD"][1], 1)
#                     qk = max(qk_idx*all_variable_specs["QK"][1], 1)
#                     v = max(v_idx*all_variable_specs["V"][1], 1)
#                     mlp = max(mlp_idx*all_variable_specs["MLP"][1], 1)
                    
#                     dim_vec = np.array([emb,head,qk,v,mlp])
#                     latency_table[emb_idx, head_idx, qk_idx, v_idx, mlp_idx] = latency_look_up_table(dim_vec)

In [102]:
# # Define model and variables
# model = ConcreteModel()
# for var_type, var_spec in all_variable_specs.items():
#     dummies = list(range(all_variable_specs[var_type][2]))
#     setattr(model, f"{var_type}_vars", Var(dummies, domain=Binary))

# model.unique_constraint = ConstraintList()
# importance = 0
# # Define constraint and importance
# for var_type, var_spec in all_variable_specs.items():
#     cur_vars = list([getattr(model, f"{var_type}_vars")[i] for i in range(all_variable_specs[var_type][2])])
#     model.unique_constraint.add(sum(cur_vars) == 1)
#     random_importance = np.abs(np.random.randn(all_variable_specs[var_type][2])) * 100
#     importance += sum(cur_vars[i] * random_importance[i] for i in range(all_variable_specs[var_type][2]))

# model.obj = Objective(expr=importance, sense=maximize)

In [103]:
# model = ConcreteModel()
# model.x = Var(bounds=(1.0,10.0),initialize=5.0)
# model.y = Var(within=Binary)
# model.c1 = Constraint(expr=(model.x-4.0)**2 - model.x <= 50.0*(1-model.y))
# model.c2 = Constraint(expr=model.x*log(model.x)+5.0 <= 50.0*(model.y))
# model.objective = Objective(expr=model.x, sense=minimize)
# SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt') 

## Trivial GLPK Test

In [20]:
model = ConcreteModel()
# Define variables
variable_slices_by_type = {}
counter = 0
for var_type, var_spec in all_variable_specs.items():
    variable_slices_by_type[var_type] = (counter, counter+all_variable_specs[var_type][2])
    counter += all_variable_specs[var_type][2]

all_items = list(range(counter))
model.decision_vars = Var(all_items, domain=Binary)

# Define importance and constraint
importance = 0
model.group_unique_constraint = ConstraintList()

for var_type, var_spec in all_variable_specs.items():
    cur_decision_vars = [model.decision_vars[k] for k in range(variable_slices_by_type[var_type][0], variable_slices_by_type[var_type][1])]
    model.group_unique_constraint.add(sum(cur_decision_vars[i] for i in list(range(len(cur_decision_vars)))) == 1)
    random_importance = np.abs(np.random.randn(all_variable_specs[var_type][2])) * 100
    importance += sum(cur_decision_vars[i] * random_importance[i] for i in range(len(cur_decision_vars)))

model.obj = Objective(expr=importance, sense=maximize)

In [21]:
# solver = SolverFactory('mindtpy')
solver = SolverFactory('glpk')
solver.solve(model)
# results = solver.solve(model, strategy='OA', init_strategy='FP', mip_solver='glpk', nlp_solver='ipopt', tee=True) 
# results = solver.solve(model, strategy='OA', init_strategy='FP', mip_solver='glpk', nlp_solver='ipopt') 
# results = solver.solve(model) 
# results = solver.solve(model, mip_solver='glpk', nlp_solver='ipopt') 

{'Problem': [{'Name': 'unknown', 'Lower bound': 1296.39062795143, 'Upper bound': 1296.39062795143, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 202, 'Number of nonzeros': 202, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.032980918884277344}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [22]:
for var_type, var_spec in all_variable_specs.items():
    indices = list(range(variable_slices_by_type[var_type][0], variable_slices_by_type[var_type][1]))
    cur_decision_vars = [model.decision_vars[k] for k in indices]
    cur_decision_vars_value = [x.value for x in cur_decision_vars]
    print(var_type, cur_decision_vars_value)

EMB [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
HEAD [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
QK [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
MLP [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

## MINLP

In [36]:
latency_table.shape

(25, 13, 33, 33, 97)

In [37]:
all_variable_specs["EMB"]

[768, 32, 25]

In [67]:
model = ConcreteModel()
# Define variables
variable_slices_by_type = {}
counter = 0
for var_type, var_spec in all_variable_specs.items():
    variable_slices_by_type[var_type] = (counter, counter+all_variable_specs[var_type][2])
    counter += all_variable_specs[var_type][2]

all_items = list(range(counter))
model.decision_vars = Var(all_items, domain=Binary)

# Define importance and constraint
importance = 0
model.group_unique_constraint = ConstraintList()

for var_type, var_spec in all_variable_specs.items():
    cur_decision_vars = [model.decision_vars[k] for k in range(variable_slices_by_type[var_type][0], variable_slices_by_type[var_type][1])]
    model.group_unique_constraint.add(sum(cur_decision_vars[i] for i in list(range(len(cur_decision_vars)))) == 1)
    random_importance = np.abs(np.random.randn(all_variable_specs[var_type][2])) * 100
    importance += sum(cur_decision_vars[i] * random_importance[i] for i in range(len(cur_decision_vars)))

# Add latency constraint
emb_vectors = np.array([model.decision_vars[k] for k in range(variable_slices_by_type["EMB"][0], variable_slices_by_type["EMB"][1])])
head_vectors = np.array([model.decision_vars[k] for k in range(variable_slices_by_type["HEAD"][0], variable_slices_by_type["HEAD"][1])])
qk_vectors = np.array([model.decision_vars[k] for k in range(variable_slices_by_type["QK"][0], variable_slices_by_type["QK"][1])])
v_vectors = np.array([model.decision_vars[k] for k in range(variable_slices_by_type["V"][0], variable_slices_by_type["V"][1])])
mlp_vectors = np.array([model.decision_vars[k] for k in range(variable_slices_by_type["MLP"][0], variable_slices_by_type["MLP"][1])])

# grid_emb, grid_head, grid_qk, grid_v, grid_mlp = np.meshgrid(emb_vectors, head_vectors, qk_vectors, v_vectors, mlp_vectors, indexing='ij')
# T = grid_emb * grid_head * grid_qk * grid_v * grid_mlp
# T = np.tensordot(np.tensordot(np.tensordot(np.tensordot(emb_vectors, head_vectors, axes=0), qk_vectors, axes=0), v_vectors, axes=0), mlp_vectors, axes=0)
T1 = np.tensordot(emb_vectors, mlp_vectors, axes=0)
T2 = np.tensordot(np.tensordot(emb, v_vectors, axes=0), mlp_vectors, axes=0)
latency_expr = sum(T * latency_table)
model.latency_constraint = Constraint(expr=latency_expr <= capacity)
model.obj = Objective(expr=importance, sense=maximize)

In [61]:
# solver = SolverFactory('mindtpy')
solver = SolverFactory('glpk')
solver.solve(model)
# results = solver.solve(model, strategy='OA', init_strategy='FP', mip_solver='glpk', nlp_solver='ipopt', tee=True) 
# results = solver.solve(model, strategy='OA', init_strategy='FP', mip_solver='glpk', nlp_solver='ipopt') 
# results = solver.solve(model) 
# results = solver.solve(model, mip_solver='glpk', nlp_solver='ipopt') 

{'Problem': [{'Name': 'unknown', 'Lower bound': 1308.26414625436, 'Upper bound': 1308.26414625436, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 202, 'Number of nonzeros': 202, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.06783676147460938}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [41]:
print(latency_table.shape)

(25, 13, 33, 33, 97)


In [50]:
emb_vectors = np.random.randn(25, 1)

In [51]:
emb_vectors.shape

(25, 1)

In [52]:
res = np.tensordot(emb_vectors, latency_table, axes=0)

In [53]:
res.shape

(25, 1, 25, 13, 33, 33, 97)

In [63]:
import numpy as np

# Initialize your vectors (replace with your actual data)
a = np.random.rand(25)
b = np.random.rand(13)
c = np.random.rand(33)
d = np.random.rand(33)
e = np.random.rand(97)

# Create the meshgrid for each vector
# Note: `numpy.meshgrid` creates coordinate matrices, so we need to use `indexing='ij'` for multi-dimensional grids
grid_a, grid_b, grid_c, grid_d, grid_e = np.meshgrid(a, b, c, d, e, indexing='ij')

# Calculate the tensor T
T = grid_a * grid_b * grid_c * grid_d * grid_e

# T now has the shape (25, 13, 33, 33, 97)
print(T.shape)

(25, 13, 33, 33, 97)


In [56]:
T[13, 6, 7, 8, 10]

0.029128482823468138

In [57]:
a[13]*b[6]*c[7]*d[8]*e[10]

0.029128482823468138

In [58]:
a.shape

(25,)

In [64]:
import numpy as np

# Initialize your vectors (replace with your actual data)
a = np.random.rand(25)
b = np.random.rand(13)
c = np.random.rand(33)
d = np.random.rand(33)
e = np.random.rand(97)

# Create the meshgrid for each vector
# We need to create a grid that has the same dimensionality as each vector
T = np.tensordot(np.tensordot(np.tensordot(np.tensordot(a, b, axes=0), c, axes=0), d, axes=0), e, axes=0)

# T now has the shape (25, 13, 33, 33, 97)
print(T.shape)

(25, 13, 33, 33, 97)
